In [2]:
#importing the necessary libraries
import numpy as np
import pandas as pd
import warnings
warnings.warn('Make sure you are in the PyViz environment')
#import plotly as plt
import os
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import requests

C:\Users\andre\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Make sure you are in the PyViz environment
  """


In [3]:
def load():
    #load dotenv in to call api keys with
    load_dotenv()

    #Load Alpaca keys
    warnings.warn('Make sure your api keys are renamed to fit this function or alter the function to fit your names')
    alpaca_public = os.getenv('ALPACA_API_KEY')
    alpaca_secret = os.getenv('ALPACA_SECRET_KEY')
    api = tradeapi.REST(alpaca_public, alpaca_secret, api_version='v2')

    return api
    

def data_pull(api):

    #initialize stock choice
    assets = api.list_assets()
    tick_list = []
    tickers = ['F', 'AAPL','ENPH','WMT','KO']
    #Try to call the historical quote of the stock
    for ticker in tickers:
        try:
            #print(ticker)
            df = api.alpha_vantage.historic_quotes(ticker, adjusted=True, output_format='pandas')
            df.columns = [f"{ticker} Open", f"{ticker} High", f"{ticker} Low", f"{ticker} Close", f"{ticker} Adjusted Close", f"{ticker} Volume", f"{ticker} Dividend Amount", f"{ticker} Split Coefficient"]
            tick_list.append(df)
        except:
            if ticker in assets:
                raise Exception('The ticker name is right, but AlphaVantage cannot pull its data, make sure the asset is tradeable')
            else:
                raise Exception('That is not a proper ticker symbol')
                
    data = pd.concat(tick_list, axis='columns', join='inner')
    return data
    

def data_clean(data):
    
    #drop unnecessary data
    data.drop([col for col in data.columns if 'Adjusted' in col], axis=1, inplace=True)
    data.drop([col for col in data.columns if 'Dividend' in col], axis=1, inplace=True)
    data.drop([col for col in data.columns if 'Split' in col], axis=1, inplace=True)
    data.drop([col for col in data.columns if 'High' in col], axis=1, inplace=True)
    data.drop([col for col in data.columns if 'Low' in col], axis=1, inplace=True)
    
    #rename the columns something that is actually understandable
    #data.columns = ['Ford Open','Ford Close','Ford Volume','Apple Open','Apple Close','Apple Volume']
    return data


In [4]:
api = load()
data = data_pull(api)
cleaned = data_clean(data)
cleaned

C:\Users\andre\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Make sure your api keys are renamed to fit this function or alter the function to fit your names
  


,F Open,F Close,F Volume,AAPL Open,AAPL Close,AAPL Volume,ENPH Open,ENPH Close,ENPH Volume,WMT Open,WMT Close,WMT Volume,KO Open,KO Close,KO Volume
date,,,,,,,,,,,,,,,
2020-07-02,6.10,6.050,67243751.0,367.85,364.11,28814516.0,49.61,48.86,3813572.0,120.09,119.21,5892639.0,45.29,44.88,15108082.0
2020-07-01,6.10,5.980,62248154.0,365.12,364.11,27690454.0,47.53,49.06,3227563.0,119.45,119.69,6579844.0,44.93,44.82,14321742.0
2020-06-30,5.98,6.080,61833037.0,360.08,364.80,35055821.0,47.68,47.57,2940539.0,119.22,119.78,6834631.0,44.25,44.68,23675721.0
2020-06-29,5.91,6.010,58135203.0,353.25,361.78,32661519.0,46.01,47.45,3467501.0,118.50,119.06,5494442.0,44.00,44.36,17596391.0
2020-06-26,6.04,5.910,118309845.0,364.41,353.63,51314211.0,45.88,45.30,13892939.0,119.60,118.32,8993309.0,44.92,43.57,21944136.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-04-05,12.38,12.470,37939500.0,626.98,633.68,22903500.0,7.48,7.43,104300.0,60.20,60.67,6528700.0,73.28,73.47,5765700.0
2012-04-04,12.46,12.505,51505000.0,624.35,624.31,20463600.0,7.77,7.45,165100.0,60.53,60.26,10851700.0,73.29,73.46,6243700.0
2012-04-03,12.71,12.640,97034200.0,627.30,629.32,29805700.0,7.75,7.64,316600.0,61.14,60.65,11180100.0,73.67,73.76,6494600.0


In [5]:
def strategy(cleaned):
    #Create a new dataframe of only the closing values
    df = cleaned.loc[:,[i for i in cleaned.columns if i.endswith('Close')]]

    #Rename the columns as only the ticker (just drop the word 'close')
    df.columns = df.columns.str.replace(' Close','')

    #Loop through the columns and create a percentage change column
    for col in df.columns:
        ticker = col

        df[f"{ticker} % Change"] = df[col].pct_change() * 100

    #return df
#     init an entry signal as 0.0 to use as it loops over the data 
    df['Entry'] = 0.0
    df['Entry'] = np.where(df['F % Change'] <= -2.0, 1.0, 0.0)
    df['Exit'] = 0.0
    df['Exit'] = np.where(df['F % Change'] >= 2.0, 1.0, 0.0)
    df['Signal'] = df['Entry'] - df['Exit']
    return df

In [6]:
strat = strategy(cleaned)
strat

,F,AAPL,ENPH,WMT,KO,F % Change,AAPL % Change,ENPH % Change,WMT % Change,KO % Change,Entry,Exit,Signal
date,,,,,,,,,,,,,
2020-07-02,6.050,364.11,48.86,119.21,44.88,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2020-07-01,5.980,364.11,49.06,119.69,44.82,-1.157025,0.000000,0.409333,0.402651,-0.133690,0.0,0.0,0.0
2020-06-30,6.080,364.80,47.57,119.78,44.68,1.672241,0.189503,-3.037097,0.075194,-0.312361,0.0,0.0,0.0
2020-06-29,6.010,361.78,47.45,119.06,44.36,-1.151316,-0.827851,-0.252260,-0.601102,-0.716204,0.0,0.0,0.0
2020-06-26,5.910,353.63,45.30,118.32,43.57,-1.663894,-2.252750,-4.531085,-0.621535,-1.780884,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-04-05,12.470,633.68,7.43,60.67,73.47,2.045827,-0.400798,6.446991,0.898054,0.823384,0.0,1.0,-1.0
2012-04-04,12.505,624.31,7.45,60.26,73.46,0.280674,-1.478664,0.269179,-0.675787,-0.013611,0.0,0.0,0.0
2012-04-03,12.640,629.32,7.64,60.65,73.76,1.079568,0.802486,2.550336,0.647195,0.408386,0.0,0.0,0.0


In [60]:
def backtest(strat):
    backtest_df = strat[['F', 'Entry', 'Exit', 'Signal']].copy()
    
    initial_capital = float(100000)
    
#     share_size = 1000
    
#     position = share_size * backtest_df['Signal']
    
    cash = initial_capital - (position * backtest_df['F'])
    
    backtest_df['Position'] = 1000
    
    
    
    

    return backtest_df

In [61]:
backtest(strat)

,F,Entry,Exit,Signal,Position
date,,,,,
2020-07-02,6.050,0.0,0.0,0.0,1000
2020-07-01,5.980,0.0,0.0,0.0,1000
2020-06-30,6.080,0.0,0.0,0.0,1000
2020-06-29,6.010,0.0,0.0,0.0,1000
2020-06-26,5.910,0.0,0.0,0.0,1000
...,...,...,...,...,...
2012-04-05,12.470,0.0,1.0,-1.0,1000
2012-04-04,12.505,0.0,0.0,0.0,1000
2012-04-03,12.640,0.0,0.0,0.0,1000
